In [1]:
'''
Data Downloaded from
"https://www.data.gov.in/"

Example Link:

Share Datasets in ICTD Drive
'''


'\nData Downloaded from\n"https://www.data.gov.in/"\n'

In [2]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
input_dir = "Karnataka_Datasets/Across/Kharif/Bidar/" #directory where XML is Different
output_dir = input_dir  # or change to another directory if needed
output_csv="Karnataka_Datasets/Across/Kharif/Bidar/all_data.csv"

In [4]:
all_rows = []

for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        file_path = os.path.join(input_dir, file_name)
        try:
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Detect namespace
            ns = {'ns': root.tag.split('}')[0].strip('{')}

            for entry in root.findall('ns:output', ns):
                row = {}
                for elem in entry:
                    tag = elem.tag.split('}')[-1]
                    row[tag] = elem.text
                all_rows.append(row)

        except ET.ParseError as e:
            print(f"❌ Skipping file {file_name}: ParseError - {e}")
        except Exception as e:
            print(f"⚠️ Error in file {file_name}: {e}")

# Convert to DataFrame
if all_rows:
    df = pd.DataFrame(all_rows)
    df.to_csv(output_csv, index=False)
    print(f"✅ Done: Saved {len(df)} rows to {output_csv}")
else:
    print("⚠️ No valid XML data found.")


✅ Done: Saved 343910 rows to Karnataka_Datasets/Across/Kharif/Bidar/all_data.csv


# REMOVE UNNECESSARY ROWS

In [8]:
import pandas as pd
import os

# === Load CSV ===
input_csv = "Karnataka_Datasets/Across/Kharif/Begalavi/Begalavi.csv"
df = pd.read_csv(input_csv)

# === Step 1: Filter Season to 'kharif' (case-insensitive) ===
df = df[df['Season'].astype(str).str.strip().str.lower() == 'kharif']

# === Step 2: Remove rows with invalid Cropname ===
invalid_keywords = ['harvest', 'fallow', 'na land']
# Drop rows where Cropname is NaN
df = df.dropna(subset=['Cropname'])

# Strip whitespace, convert to lowercase for filtering
cropname_clean = df['Cropname'].str.strip().str.lower()

# Drop rows with blank crop names or unwanted keywords
df = df[~cropname_clean.isin([''])]
df = df[~cropname_clean.str.contains('|'.join(invalid_keywords), na=False)]

# === Step 3: Split data by unique Years and deduplicate each ===
output_dir = os.path.dirname(input_csv)  # Save in same directory

for year in df['Years'].dropna().unique():
    year_df = df[df['Years'] == year]
    year_df = year_df.drop_duplicates(subset=['Latitude', 'Longtitude'])
    output_path = os.path.join(output_dir, f"data_{year}.csv")
    year_df.to_csv(output_path, index=False)

print("✅ Done: Filtered, split by year, and removed duplicates.")


✅ Done: Filtered, split by year, and removed duplicates.
